In [1]:
import os
os.environ["DASHSCOPE_API_KEY"] = os.getenv("DASHSCOPE_API_KEY")  # 阿里云
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

from langchain.indexes import VectorstoreIndexCreator
from langchain_community.embeddings import DashScopeEmbeddings

/Users/zhaoyang/opt/anaconda3/envs/langchain_RAG_3.10/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
# 初始化DashScope Embeddings
embeddings = DashScopeEmbeddings(
    model="text-embedding-v1",
    dashscope_api_key= os.getenv("DASHSCOPE_API_KEY")
)
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_loaders([loader])

/Users/zhaoyang/opt/anaconda3/envs/langchain_RAG_3.10/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [4]:
llm = ChatOpenAI(
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    openai_api_key=os.getenv("DASHSCOPE_API_KEY"),
    model_name="qwen-plus",    # 模型名称
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

/var/folders/2p/9b5w6pj56_s1vvr_y6r4cmqc0000gn/T/ipykernel_99822/3249136602.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [5]:
from langchain.evaluation.qa import QAGenerateChain
example_gen_chain = QAGenerateChain.from_llm(llm)
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

new_list = [item['qa_pairs'] for item in new_examples]
display(new_list)
examples += new_list
print(len(examples))
qa.invoke(examples[0]["query"])

/Users/zhaoyang/opt/anaconda3/envs/langchain_RAG_3.10/lib/python3.10/site-packages/langchain/chains/llm.py:370: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/zhaoyang/opt/anaconda3/envs/langchain_RAG_3.10/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:174: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


[{'query': "What features contribute to the comfort and durability of the Women's Campside Oxfords, and what is the approximate weight per pair?",
  'answer': "The Women's Campside Oxfords feature a super-soft canvas material for a broken-in feel, a comfortable EVA innersole with antimicrobial odor control (Cleansport NXT®), moderate arch contouring, and an EVA foam midsole for cushioning and support. The chain-tread-inspired molded rubber outsole adds durability. The approximate weight of each pair is 1 pound 1 ounce."},
 {'query': 'What are the dimensions of the medium-sized Recycled Waterhog Dog Mat, and what is one of its notable construction features?',
  'answer': 'The medium-sized Recycled Waterhog Dog Mat has dimensions of 22.5" x 34.5". One of its notable construction features is its 24 oz. polyester fabric made from 94% recycled materials, along with a rubber backing.'},
 {'query': "What features of the Infant and Toddler Girls' Coastal Chill Swimsuit provide sun protection a

7


> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Yes, the Cozy Comfort Pullover Set has side pockets. The pull-on pants included in the set feature side pockets.'}

In [6]:
import langchain
langchain.debug = True
qa.invoke(examples[0]["query"])
#langchain.debug = False
predictions = qa.apply(examples)
display(predictions)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-

/var/folders/2p/9b5w6pj56_s1vvr_y6r4cmqc0000gn/T/ipykernel_99822/3837145942.py:5: LangChainDeprecationWarning: The method `Chain.apply` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~batch` instead.
  predictions = qa.apply(examples)


[llm/end] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain > llm:ChatOpenAI] [1.09s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Yes, the Cozy Comfort Pullover Set has side pockets. The description mentions that the pull-on pants include side pockets.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Yes, the Cozy Comfort Pullover Set has side pockets. The description mentions that the pull-on pants include side pockets.",
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 25,
                "prompt_tokens": 809,
                "total_tokens": 834,
      

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes',
  'result': 'Yes, the Cozy Comfort Pullover Set has side pockets. The description mentions that the pull-on pants include side pockets.'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection',
  'result': 'The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the **DownTek collection**.'},
 {'query': "What features contribute to the comfort and durability of the Women's Campside Oxfords, and what is the approximate weight per pair?",
  'answer': "The Women's Campside Oxfords feature a super-soft canvas material for a broken-in feel, a comfortable EVA innersole with antimicrobial odor control (Cleansport NXT®), moderate arch contouring, and an EVA foam midsole for cushioning and support. The chain-tread-inspired molded rubber outsole adds durability. The approximate weight of each pair is 1 pound 1 ounce.",
  'result':

In [7]:
from langchain.evaluation.qa import QAEvalChain
llm = ChatOpenAI(
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    openai_api_key=os.getenv("DASHSCOPE_API_KEY"),
    model_name="qwen-plus",	# 模型名称
    temperature=0.0
)
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs  = eval_chain.evaluate(examples,predictions)
display(graded_outputs)
display(graded_outputs[0]['results'])

for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

[chain/start] [chain:QAEvalChain] Entering Chain run with input:
{
  "input_list": [
    {
      "query": "Do the Cozy Comfort Pullover Set        have side pockets?",
      "answer": "Yes",
      "result": "Yes, the Cozy Comfort Pullover Set has side pockets. The description mentions that the pull-on pants include side pockets."
    },
    {
      "query": "What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?",
      "answer": "The DownTek collection",
      "result": "The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the **DownTek collection**."
    },
    {
      "query": "What features contribute to the comfort and durability of the Women's Campside Oxfords, and what is the approximate weight per pair?",
      "answer": "The Women's Campside Oxfords feature a super-soft canvas material for a broken-in feel, a comfortable EVA innersole with antimicrobial odor control (Cleansport NXT®), moderate arch contouring, and an EVA foam midsole for cushioning a

/Users/zhaoyang/opt/anaconda3/envs/langchain_RAG_3.10/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:174: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


[{'results': 'CORRECT'},
 {'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: **CORRECT**'},
 {'results': "GRADE: **CORRECT**  \n\nThe student's answer accurately identifies key features (weather-blocking gaiters and side zips) that make the pants versatile for year-round outdoor use. The explanation of TEK O2 technology, while phrased differently, aligns with the true answer by correctly describing its role in enhancing breathability while maintaining waterproof protection. There are no factual inaccuracies, and the additional details do not conflict with the true answer."}]

'CORRECT'

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set has side pockets. The description mentions that the pull-on pants include side pockets.
Predicted Grade: CORRECT

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the **DownTek collection**.
Predicted Grade: GRADE: CORRECT

Example 2:
Question: What features contribute to the comfort and durability of the Women's Campside Oxfords, and what is the approximate weight per pair?
Real Answer: The Women's Campside Oxfords feature a super-soft canvas material for a broken-in feel, a comfortable EVA innersole with antimicrobial odor control (Cleansport NXT®), moderate arch contouring, and an EVA foam midsole for cushioning and support. The chain-tread-inspired molded rubber outsole adds d